In [15]:
import requests
from bs4 import BeautifulSoup as bs
import bs4
import json

title_type = 'tv_movie,tv_series'
release_date = '1980-01-01,2020-12-31'
genres = 'comedy,fantasy'
user_raiting = '8.0,10.0'
countries = 'us'


body_url = 'https://www.imdb.com/search/title/?title_type={0}&release_date={1}&user_rating={2}&genres={3}&countries={4}'
url = body_url.format(title_type,release_date,user_raiting,genres,countries)

logs = open('logs.txt', 'w')   #файл для записи информации о процессе работы и ошибках
logs.write('Собрана ссылка ' + url + '\n')

try:
    response = requests.get(url)
except Exception:
    logs.write('Невозможно открыть ссылку\n')
    
logs.write('Открыта ссылка: ' + url +'\n')

html = response.text
html = bs(html, 'lxml')
logs.write('Ответ преобразован в BS HTML\n')

inf = html.find("div", {"class": "desc"})
count = inf.find("span")

try:
    count = int(str(count.text).split()[2])
except Exception:
    count = int(str(count.text).split()[0])
logs.write("Найдено " + str(count) + " фильмов\n")
number = 0

page = 0
if (count % 10 != 0):
    pages = int(count/50)+1
else:
    pages = int(count/50)
    
dataList = []
    
while (page < pages):
    films = html.find("div", {"class": "lister-list"})
    
    for film in films.findAll("div", {"class": "lister-item mode-advanced"}):
        number += 1
        
        header = film.find("h3")
        name = header.find("a").text
        logs.write('Анализ фильма № ' + str(number) + ' '+ name + '\n')
        link = header.find("a")
        link = link.get('href')
        link = 'https://www.imdb.com' + link

        genre = film.find("span", {"class": "genre"})
        genre = genre.text
        f = genre.find('\n')
        if (f != -1):
            genre = genre[f+1:]
        n = genre.find('  ')
        if (n != -1):
            genre = genre[:n]

        rating = film.find("div", {"class": "inline-block ratings-imdb-rating"})
        rating = rating.find("strong").text

        starsp = film.find("p", {"class": ""})
        starsa = starsp.findAll("a")
        stars = []
        for s in starsa:
            stars.append(s.text)

        logs.write('Общие сведения собраны\n')

        resp = requests.get(link)
        html_film = resp.text
        html_film = bs(html_film, 'lxml')

        sub = html_film.find("div", {"class": "subtext"})
        typeof = sub.findAll("a")[-1].text

        detail_block = html_film.find("div", {"class": "article","id": "titleDetails"})

        detail_text = str(detail_block)
        blocks = detail_text.split('<hr/>')
        block_list = []
        for i in blocks:
            block  = bs(i, 'lxml')
            block_list.append(block)
        details = []
        h2 = block_list[0].find("h2")
        if (h2.text == "Details"):

            lines = block_list[0].findAll("div", {"class": "txt-block"})
            lst = []
            for line in lines:
                lst.append(line.find("span", {"class": "see-more inline"}))
                sp = line.find("span", {"class": "see-more inline"})
                seemore = False

                if (sp != None):
                    fa = sp.find("a")
                    if (sp.find("a")!= None):
                        if (fa.text == "See more"):
                            seemore = True

                h4 = line.find("h4", {"class": "inline"})
                a = line.find("a")
                spl = line.text.split('\n')

                href = a.get('href')
                lin = []
                dictline = {}
                for e in range(len(spl)):
                    setspace = ('', ' ', '  ', '   ', '    ', '     ', '      ')
                    if (spl[e] not in setspace):                    
                        if (spl[e] != "See more\xa0»"):
                            lin.append(spl[e])
                if(len(lin)>1):
                    dictline.update({lin[0]:lin[1]})
                else:
                    lin = lin[0].split(': ')
                    dictline.update({lin[0]:lin[1]})
                if(not seemore):
                    dictline.update({'Ссылка':'https://www.imdb.com' + href})

                details.append(dictline)

        specs = []
        box = []
        for e in range(len(block_list)):
            bl = block_list[e]
            h3 = bl.find("h3", {"class": "subheading"})

            if (h3 != None):
                if (h3.text == "Box Office"):
                    lines = bl.findAll("div", {"class": "txt-block"})
                    for line in lines:
                        if (line.text != None):
                            sign = line.text.find(':')
                            if (sign != -1):
                                k = line.text[0:sign]
                                v = line.text[sign+2:]
                                n = k.find('\n')
                                k = k[n+1:]
                                n = v.find('\n')
                                v = v[:n]
                                box.append({k:v})
                elif (h3.text == "Technical Specs"):
                    lines = bl.findAll("div", {"class": "txt-block"})
                    for line in lines:
                        if (line.text != None):
                            sign = line.text.find(':')
                            if (sign != -1):
                                k = line.text[0:sign]
                                v = line.text[sign+2:]
                                n = k.find('\n')
                                k = k[n+1:]
                                n = v.find('\n')
                                v = v[:n]
                                specs.append({k:v})
                                
        logs.write("Детали записаны\n")
        
        if (len(box) == 0):
            box.append("No info")
            
        info = {"Название": name,
                "Жанры": genre,
                "Рейтинг": rating,
                "Топ каста": stars,
                "Тип": typeof,
                "Details": details,
                "Box office": box,
                "Technical specs": specs}  
        
        dataList.append(info)
        print(int(number/(count+50*page)*100), '%')
        
    if (count/50 > 1):
        link = ''
        next_url = inf.find("a", {"class": "lister-page-next next-page"})
        if (next_url != None):
            link = next_url.get('href')
            next_link = 'https://www.imdb.com' + link

            response = requests.get(next_link)
            logs.write('Открыта ссылка ' + next_link + ' ')

            html = response.text
            html = bs(html, 'lxml')
            logs.write('Ответ преобразован в BS HTML\n')  
            inf = html.find("div", {"class": "desc"})
        count -= 50
    page+=1
    
logs.close()    
    
filename = 'output.json'

with open(filename, 'w', encoding='utf-8') as file:
    file.write(json.dumps(dataList, indent = 4, ensure_ascii=False))#, sort_keys=True

0 %
1 %
2 %
3 %
4 %
5 %
6 %
7 %
7 %
8 %
9 %
10 %
11 %
12 %
13 %
14 %
14 %
15 %
16 %
17 %
18 %
19 %
20 %
21 %
21 %
22 %
23 %
24 %
25 %
26 %
27 %
28 %
28 %
29 %
30 %
31 %
32 %
33 %
34 %
35 %
35 %
36 %
37 %
38 %
39 %
40 %
41 %
42 %
42 %
43 %
44 %
45 %
46 %
47 %
48 %
49 %
50 %
50 %
51 %
52 %
53 %
54 %
55 %
56 %
57 %
57 %
58 %
59 %
60 %
61 %
62 %
63 %
64 %
64 %
65 %
66 %
67 %
68 %
69 %
70 %
71 %
71 %
72 %
73 %
74 %
75 %
76 %
77 %
78 %
78 %
79 %
80 %
81 %
82 %
83 %
84 %
85 %
85 %
86 %
87 %
88 %
89 %
90 %
91 %
92 %
92 %
93 %
94 %
95 %
96 %
97 %
98 %
99 %
100 %
